In [ ]:
import gym
import numpy as np
from random import randint
from MountainCarEnv import MountainCarEnv

In [ ]:
env = MountainCarEnv(render_mode="human")

In [ ]:
def discretizar(valor):
    aux = ((valor - env.observation_space.low) / (env.observation_space.high - env.observation_space.low)) * 20
    return tuple(aux.astype(np.int32))

In [ ]:
discretizar(env.reset())

In [ ]:
pos_space = np.linspace(-5, 5, 10)
vel_space = np.linspace(-3, 3, 2)
pos_space

In [ ]:
def get_state(obs):
    pos, vel = obs
    pos_bin = np.digitize(pos, pos_space)
    vel_bin = np.digitize(vel, vel_space)
    return pos_bin, vel_bin

In [ ]:
estado = get_state(np.array([-0.4, 0.2]))
estado

In [ ]:
q_table = np.random.uniform(low=-1, high=1, size=(20, 20, 3))

In [ ]:

tasa_aprendizaje = 0.1
factor_descuento = 0.95
episodios = 5000
listado_recompensas = []
for episodio in range(episodios):
    estado = discretizar(env.reset())
    done = False
    recompensa_total = 0
    while not done:
        if randint(0, 10) > 2:
            accion = np.argmax(q_table[estado])
        else:
            accion = randint(0, 2)
        
        nuevo_estado, recompensa, done, _ = env.step(accion)
        q_table[estado][accion] = q_table[estado][accion] + tasa_aprendizaje * (recompensa + factor_descuento * np.max(q_table[discretizar(nuevo_estado)]) - q_table[estado][accion])
        estado = discretizar(nuevo_estado)
        recompensa_total += recompensa
        if (episodio + 1) % 500 == 0:
            env.render()
    listado_recompensas.append(recompensa_total)
    if (episodio + 1) % 100 == 0:
        print(f"Episodio: {episodio + 1} -  Recompensa: {np.mean(listado_recompensas)}")
    env.close()